In [187]:
import os
import json
import pandas as pd
import numpy as np

# Configuration
LOG_BASE_PATH = "/home/jihwan/transmiter_aaai26/transmiter/logs/Transfer_ViT-B-16->ViT-L-14"
DATASETS = ["EuroSAT", "DTD", "Food101", "Pets", "Aircraft", "Flowers", "UCF", "Caltech", "Cars", "SUN397"]
SEEDS = [1, 2, 3]

def parse_config_file(config_path):
    """Parse the config.json file and extract base_acc and novel_acc."""
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        
        base = config.get('base_acc', None)
        novel = config.get('novel_acc', None)
        return base, novel
    except (FileNotFoundError, json.JSONDecodeError):
        return None, None

def harmonic_mean(base, novel):
    """Calculate Harmonic Mean of Base and Novel."""
    if base + novel == 0:
        return 0
    return 2 * base * novel / (base + novel)

# Collect results
results = {}

for dataset in DATASETS:
    base_scores = []
    novel_scores = []
    
    for seed in SEEDS:
        config_path = os.path.join(LOG_BASE_PATH, dataset, str(seed), "promptsrc_logits", "config.json")
        base, novel = parse_config_file(config_path)
        
        if base is not None and novel is not None:
            base_scores.append(base)
            novel_scores.append(novel)
            print(f"{dataset} Seed {seed}: Base={base:.2f}%, Novel={novel:.2f}%")
        else:
            print(f"{dataset} Seed {seed}: Config file not found or parse error")
    
    if base_scores and novel_scores:
        avg_base = np.mean(base_scores)
        avg_novel = np.mean(novel_scores)
        avg_hm = harmonic_mean(avg_base, avg_novel)
        results[dataset] = {
            'Base': avg_base,
            'Novel': avg_novel,
            'HM': avg_hm
        }

print("\n" + "="*50)

EuroSAT Seed 1: Base=93.05%, Novel=88.03%
EuroSAT Seed 2: Base=93.24%, Novel=87.00%
EuroSAT Seed 3: Base=91.90%, Novel=81.51%
DTD Seed 1: Base=82.87%, Novel=68.72%
DTD Seed 2: Base=83.22%, Novel=71.26%
DTD Seed 3: Base=81.83%, Novel=67.63%
Food101 Seed 1: Base=94.12%, Novel=94.91%
Food101 Seed 2: Base=94.31%, Novel=94.83%
Food101 Seed 3: Base=94.16%, Novel=94.36%
Pets Seed 1: Base=96.01%, Novel=93.62%
Pets Seed 2: Base=96.23%, Novel=98.94%
Pets Seed 3: Base=95.96%, Novel=98.88%
Aircraft Seed 1: Base=45.92%, Novel=45.47%
Aircraft Seed 2: Base=45.26%, Novel=43.55%
Aircraft Seed 3: Base=44.48%, Novel=41.63%
Flowers Seed 1: Base=96.58%, Novel=81.42%
Flowers Seed 2: Base=97.06%, Novel=81.06%
Flowers Seed 3: Base=98.01%, Novel=83.12%
UCF Seed 1: Base=86.87%, Novel=82.91%
UCF Seed 2: Base=86.66%, Novel=83.29%
UCF Seed 3: Base=85.83%, Novel=82.15%
Caltech Seed 1: Base=98.64%, Novel=96.40%
Caltech Seed 2: Base=98.58%, Novel=95.41%
Caltech Seed 3: Base=98.58%, Novel=97.60%
Cars Seed 1: Base=80.3

In [188]:
# Create DataFrame and display table
df = pd.DataFrame(results).T
df = df.round(2)

# Reorder columns
df = df[['Base', 'Novel', 'HM']]

# Add average row
avg_row = pd.DataFrame({
    'Base': [df['Base'].mean()],
    'Novel': [df['Novel'].mean()],
    'HM': [df['HM'].mean()]
}, index=['Average'])

df = pd.concat([df, avg_row])
df = df.round(2)

print("\nTransfer_ViT-B-16->ViT-L-14 Results (Seed Average)")
print("="*50)
display(df)


Transfer_ViT-B-16->ViT-L-14 Results (Seed Average)


,Base,Novel,HM
EuroSAT,92.73,85.51,88.98
DTD,82.64,69.20,75.33
Food101,94.20,94.70,94.45
Pets,96.07,97.15,96.60
Aircraft,45.22,43.55,44.37
Flowers,97.21,81.87,88.88
UCF,86.45,82.78,84.58
Caltech,98.60,96.47,97.52
Cars,80.38,83.90,82.10
SUN397,82.53,80.90,81.71
